In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os

In [2]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
mixed_precision.set_global_policy("mixed_float16")

TensorFlow version: 2.18.0
Num GPUs Available: 1


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
USE_META = True
META_CNT = 5

PATH_WEIGHTS = 'drive/MyDrive/v1.weights.h5'

IMG_SIZE = (224, 224)
IMG_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], 3)
USE_FINE_TUNE = True
FINE_TUNE_AT = -50
BATCH_SIZE = 32
EPOCHS = 5

In [5]:
dataset, info = tfds.load(
    "food101",
    with_info=True,
    as_supervised=True
)

# Берем 80% от train для обучения, 20% для валидации
train_dataset = dataset['train'].take(60700)  # ~80% от 75750
validation_dataset = dataset['train'].skip(60700)

# Оригинальный validation используем как test
test_dataset = dataset['validation']

# stat
num_classes = info.features['label'].num_classes
class_names = info.features['label'].names

if USE_META:
    print(f'Общее кол-во классов: {num_classes}')
    print(f'Названия: {class_names[:META_CNT]}')


Общее кол-во классов: 101
Названия: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']


In [6]:
with open('drive/MyDrive/classes.txt', 'w') as f:
    f.write(",".join(class_names))

In [7]:
def prepare_dataset(
        dataset,
        batch_size=BATCH_SIZE,
        img_size=IMG_SIZE,
        shuffle=False,
        augment=False
):
    preprocess_fn = tf.keras.applications.resnet50.preprocess_input

    def format_image(image, label):
        image = tf.image.resize(image, img_size)
        if augment:
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_flip_up_down(image)
            # image = tf.image.random_brightness(image, max_delta=0.2)
            # image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
            # image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
        image = preprocess_fn(image)
        return image, label

    ds = dataset.map(format_image, num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1024)

    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [8]:
train_resnet = prepare_dataset(
    dataset=train_dataset,
    shuffle=True,
    augment=True
)

test_resnet = prepare_dataset(
    dataset=test_dataset
)

valid_resnet = prepare_dataset(
    dataset=validation_dataset
)

In [9]:
def build_resnet50(
        num_classes,
        input_shape=IMG_SHAPE,
        fine_tune_at=FINE_TUNE_AT
):

    base_model = ResNet50(
        weights = 'imagenet',
        include_top = False,
        input_shape = input_shape
    )

    for layer in base_model.layers:
        layer.trainable = False

    if USE_FINE_TUNE:
        for layer in base_model.layers[fine_tune_at:]:
            layer.trainable = True

    model = models.Sequential([
        base_model,
        # layers.Flatten(),
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])
    return model

model_resnet50 = build_resnet50(num_classes=num_classes)
model_resnet50.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

if os.path.exists(PATH_WEIGHTS):
    model_resnet50.load_weights('drive/MyDrive/v1.weights.h5')
    print(f'Веса по пути {PATH_WEIGHTS} загружены!')
else:
    print(f'Не удалось найти веса по пути {PATH_WEIGHTS}. Обучаем с нуля')

model_resnet50.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 158 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Веса по пути drive/MyDrive/v1.weights.h5 загружены!


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 154 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │        25,957 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,138,213 (92.08 MB)

 Trainable params: 17,501,029 (66.76 MB)

 Non-trainable params: 6,637,184 (25.32 MB)

In [10]:
model_checkpoint = ModelCheckpoint(
    filepath=PATH_WEIGHTS,
    monitor='val_loss',
    save_weights_only=True,
    verbose=1
)

In [14]:
history_resnet50 = model_resnet50.fit(
    train_resnet,
    validation_data=valid_resnet,
    epochs=EPOCHS,
    callbacks=[model_checkpoint]
)

Epoch 1/5
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.8761 - loss: 0.4270
Epoch 1: saving model to drive/MyDrive/v1.weights.h5
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 382s 199ms/step - accuracy: 0.8761 - loss: 0.4270 - val_accuracy: 0.6955 - val_loss: 1.4931
Epoch 2/5
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.8856 - loss: 0.3850
Epoch 2: saving model to drive/MyDrive/v1.weights.h5
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 390s 203ms/step - accuracy: 0.8856 - loss: 0.3850 - val_accuracy: 0.6950 - val_loss: 1.5417
Epoch 3/5
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.8954 - loss: 0.3522
Epoch 3: saving model to drive/MyDrive/v1.weights.h5
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 400s 208ms/step - accuracy: 0.8954 - loss: 0.3522 - val_accuracy: 0.6891 - val_loss: 1.6116
Epoch 4/5
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.9032 - loss: 0.3247
Epoch 4: saving model to drive/MyDrive/v1.weights.h5
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 406s 211ms/step - accuracy: 0.90

In [15]:
test_loss, test_acc = model_resnet50.evaluate(test_resnet)
print(f'Тестовая точность {test_acc:.4f}\nОшибки {test_loss:.4f}')

790/790 ━━━━━━━━━━━━━━━━━━━━ 85s 107ms/step - accuracy: 0.7499 - loss: 1.2037
Тестовая точность 0.7474
Ошибки 1.2196
